In [1]:
!pip install pinecone-client cohere transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [2]:
import cohere
import pinecone
from transformers import AutoTokenizer , AutoModel
import torch

In [3]:
from pinecone import Pinecone

# Initialize Pinecone with your API key
pc = Pinecone(api_key='1e7f6c93-018b-40f4-b83a-2ebb8bcd379e')

# List all indices
indices = pc.list_indexes()

# Print the available indices
print("Available indices:", indices)


Available indices: {'indexes': [{'deletion_protection': 'disabled',
              'dimension': 768,
              'host': 'qa-bot-index-dh4rbd4.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'qa-bot-index',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [4]:
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone with your API key
pc = Pinecone(api_key='1e7f6c93-018b-40f4-b83a-2ebb8bcd379e')

# Define a serverless spec with AWS and the 'us-east-1' region
spec = ServerlessSpec(cloud='aws', region='us-east-1')

# Create the index if it doesn't already exist
index_name = 'qa-bot-index'
try:
    pc.create_index(
        name=index_name,
        dimension=768,  # Adjust dimension based on your embedding model (e.g., BERT is 768)
        metric='cosine',  # You can use 'cosine', 'dotproduct', or 'euclidean'
        spec=spec
    )
    print(f"Index '{index_name}' created successfully.")
except Exception as e:
    print(f"Error creating index: {e}")


Error creating index: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '76840f430dbb75e6e2c87f3d432273f2', 'Date': 'Fri, 20 Sep 2024 13:40:29 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}



In [5]:
#Load your documents and create embeddings using a model from Hugging Face's Transformers.

from transformers import AutoTokenizer, AutoModel
import torch

# Load a pre-trained model and tokenizer
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Example documents (replace with your actual documents)
documents = [
    "Pinecone is a vector database service.",
    "Transformers are used in NLP tasks.",
    "Question Answering systems can be built using RAG models."
]

# Function to create embeddings
def create_embeddings(documents):
    inputs = tokenizer(documents, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.numpy()

# Generate embeddings for your documents
doc_embeddings = create_embeddings(documents)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [6]:
# Initialize Pinecone with your API key
pc = Pinecone(api_key='1e7f6c93-018b-40f4-b83a-2ebb8bcd379e')

# Check available indices
indices = pc.list_indexes()
print("Available indices:", indices)

# Specify the index name
index_name = 'qa-bot-index'

# Since the index already exists, you should check its dimensions
# (assuming you have a method to check this, or refer to the index creation)
# For demonstration purposes, let's say it's correctly set to 384.

# Use existing index
index = pc.Index(index_name)


Available indices: {'indexes': [{'deletion_protection': 'disabled',
              'dimension': 768,
              'host': 'qa-bot-index-dh4rbd4.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'qa-bot-index',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [7]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load BERT tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embeddings(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the mean of the last hidden states for embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [8]:
# Define your documents as a list of strings
documents = [
    "Your document text 1",
    "Your document text 2",
    "Another example document.",
    "Yet another document for testing."
]

# Generate embeddings for each document
doc_embeddings = [get_embeddings(doc) for doc in documents]


In [9]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load BERT tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embeddings(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the mean of the last hidden states for embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

# Define your documents
documents = [
    "Your document text 1",
    "Your document text 2",
    "Another example document.",
    "Yet another document for testing."
]

# Generate embeddings for each document
doc_embeddings = [get_embeddings(doc) for doc in documents]

# Initialize Pinecone and connect to the existing index
pc = Pinecone(api_key='1e7f6c93-018b-40f4-b83a-2ebb8bcd379e')
index = pc.Index('qa-bot-index')

# Upsert documents to the index
upsert_data = [(str(i), doc_embeddings[i]) for i in range(len(doc_embeddings))]
index.upsert(upsert_data)
print("Documents upserted to the index.")


Documents upserted to the index.


In [10]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state[:, 0, :].numpy()  # Get the embeddings
    # Check the shape and convert to a list
    assert embeddings.shape[1] == 768, "Embedding dimension mismatch!"
    return embeddings.flatten().tolist()  # Return as a flat list


In [13]:
user_query = "What is the capital of France?"  # Example query
query_embedding = get_embeddings(user_query)
print("Query Embedding:", query_embedding)
print("Embedding Length:", len(query_embedding))


Query Embedding: [-0.18036594986915588, 0.22870655357837677, -0.11278809607028961, 0.019350189715623856, -0.27691930532455444, -0.15077091753482819, 0.19055922329425812, 0.42895230650901794, -0.2581484615802765, 0.08916348963975906, -0.006567666772753, -0.29160648584365845, 0.1402854174375534, 0.25588780641555786, 0.3149610757827759, -0.007998980581760406, -0.13502685725688934, 0.4224277436733246, 0.4468942880630493, 0.007590481545776129, 0.014095447957515717, 0.033567771315574646, -0.40684962272644043, -0.16937854886054993, 0.22148025035858154, -0.1037290096282959, 0.1857716292142868, -0.29301249980926514, -0.15571247041225433, -0.10356226563453674, -0.0023232398089021444, -0.03324718028306961, -0.15406857430934906, -0.3233778476715088, -0.05717826634645462, 0.024091238155961037, 0.19797438383102417, -0.02091892622411251, 0.12852218747138977, 0.1250697374343872, -0.3651815950870514, -0.07888694107532501, 0.05975720286369324, 0.04298682510852814, 0.18028667569160461, -0.588693261146545

In [14]:
import numpy as np

def normalize_vector(vec):
    norm = np.linalg.norm(vec)
    if norm == 0:
        return vec  # Handle case of zero vector
    return vec / norm

query_embedding = normalize_vector(query_embedding).tolist()  # Normalize and convert to list


In [15]:

query_embedding = get_embeddings(user_query)
print("Query Embedding:", query_embedding)
print("Embedding Length:", len(query_embedding))


Query Embedding: [-0.18036594986915588, 0.22870655357837677, -0.11278809607028961, 0.019350189715623856, -0.27691930532455444, -0.15077091753482819, 0.19055922329425812, 0.42895230650901794, -0.2581484615802765, 0.08916348963975906, -0.006567666772753, -0.29160648584365845, 0.1402854174375534, 0.25588780641555786, 0.3149610757827759, -0.007998980581760406, -0.13502685725688934, 0.4224277436733246, 0.4468942880630493, 0.007590481545776129, 0.014095447957515717, 0.033567771315574646, -0.40684962272644043, -0.16937854886054993, 0.22148025035858154, -0.1037290096282959, 0.1857716292142868, -0.29301249980926514, -0.15571247041225433, -0.10356226563453674, -0.0023232398089021444, -0.03324718028306961, -0.15406857430934906, -0.3233778476715088, -0.05717826634645462, 0.024091238155961037, 0.19797438383102417, -0.02091892622411251, 0.12852218747138977, 0.1250697374343872, -0.3651815950870514, -0.07888694107532501, 0.05975720286369324, 0.04298682510852814, 0.18028667569160461, -0.588693261146545

In [16]:
if isinstance(query_embedding, np.ndarray):
    query_embedding = query_embedding.tolist()


In [17]:
normalized_embedding = normalize_vector(query_embedding)
print("Normalized Query Embedding:", normalized_embedding)


Normalized Query Embedding: [-1.24692312e-02  1.58111600e-02 -7.79737444e-03  1.33773581e-03
 -1.91442501e-02 -1.04232392e-02  1.31739223e-02  2.96547408e-02
 -1.78465661e-02  6.16413558e-03 -4.54042216e-04 -2.01596183e-02
  9.69834554e-03  1.76902804e-02  2.17741901e-02 -5.52993170e-04
 -9.33480574e-03  2.92036785e-02  3.08951230e-02  5.24752424e-04
  9.74459979e-04  2.32063925e-03 -2.81267169e-02 -1.17096397e-02
  1.53115843e-02 -7.17109302e-03  1.28429418e-02 -2.02568202e-02
 -1.07648633e-02 -7.15956552e-03 -1.60612435e-04 -2.29847584e-03
 -1.06512159e-02 -2.23560664e-02 -3.95290255e-03  1.66549850e-03
  1.36865543e-02 -1.44618720e-03  8.88511869e-03  8.64644061e-03
 -2.52460830e-02 -5.45368740e-03  4.13119206e-03  2.97180627e-03
  1.24637508e-02 -4.06981051e-02 -1.60887710e-01 -6.89191982e-03
 -2.57513195e-02 -7.94818285e-03  6.78581990e-03  1.49791214e-02
  3.58823977e-02  2.46138153e-02 -9.70790956e-03  2.15195383e-02
 -2.42019301e-02  3.72058253e-02 -8.61889564e-03 -1.06437400e-

In [18]:
def query_index(user_query):
    query_embedding = get_embeddings(user_query).tolist()  # Ensure it's a list
    return index.query(query_embedding, top_k=5)  # Adjust top_k as needed


In [20]:
# Assuming get_embeddings function is defined and index is initialized
user_query = "What is the capital of France?"  # Example query
test_query_embedding = get_embeddings(user_query)  # Get the embedding for the query

# Make sure index is properly initialized, and Pinecone is set up
results = index.query(vector=test_query_embedding, top_k=5)
print("Results from test query:", results)


Results from test query: {'matches': [{'id': '3', 'score': 0.430343777, 'values': []},
             {'id': '0', 'score': 0.353046507, 'values': []},
             {'id': '1', 'score': 0.346308, 'values': []},
             {'id': '2', 'score': 0.209052578, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}


In [21]:
# Create a test query embedding
test_query_embedding = [0.1] * 768  # Example values; ensure the dimension matches

# Perform the query using keyword arguments
results = index.query(vector=test_query_embedding, top_k=5)

# Print the results
print("Results from test query:", results)


Results from test query: {'matches': [{'id': '1', 'score': -0.033778917, 'values': []},
             {'id': '0', 'score': -0.0353978314, 'values': []},
             {'id': '2', 'score': -0.0365567207, 'values': []},
             {'id': '3', 'score': -0.0396663211, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}


In [22]:
def get_query_embedding(user_query):
    # Example: Replace with your model's embedding function
    query_embedding = model.encode(user_query)  # Adjust based on your model
    return query_embedding


In [23]:
from transformers import BertTokenizer, BertModel
import torch

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Set the model to evaluation mode
model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [24]:
def get_query_embedding(user_query):
    # Tokenize the input
    inputs = tokenizer(user_query, return_tensors='pt', padding=True, truncation=True)

    # Get the output from the model
    with torch.no_grad():  # Disable gradient calculation
        outputs = model(**inputs)

    # Use the last hidden state (you can adjust this based on your needs)
    query_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Average pooling

    return query_embedding


In [25]:
def get_query_embedding(user_query):
    # Tokenize the input
    inputs = tokenizer(user_query, return_tensors='pt', padding=True, truncation=True)

    # Get the output from the model
    with torch.no_grad():  # Disable gradient calculation
        outputs = model(**inputs)

    # Use the last hidden state (you can adjust this based on your needs)
    query_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()  # Convert to list

    return query_embedding


In [26]:
user_query = "What is the content of document 1?"  # Example query
query_embedding = get_query_embedding(user_query)
results = index.query(vector=query_embedding, top_k=5, include_values=True, include_metadata=True)

# Print results
print("Top matching documents:")
for match in results['matches']:
    print(f"Document ID: {match['id']}, Score: {match['score']}, Metadata: {match.get('metadata', 'No metadata')}")


Top matching documents:
Document ID: 0, Score: 0.695813417, Metadata: No metadata
Document ID: 1, Score: 0.666098475, Metadata: No metadata
Document ID: 2, Score: 0.633944392, Metadata: No metadata
Document ID: 3, Score: 0.613308191, Metadata: No metadata


In [27]:
# Example documents with metadata
documents = [
    {"text": "Your document text 1", "metadata": {"title": "Doc 1", "author": "Author A"}},
    {"text": "Your document text 2", "metadata": {"title": "Doc 2", "author": "Author B"}},
    # Add more documents as needed
]

# Upsert documents with metadata
upsert_data = [(str(i), get_embeddings(doc['text']), doc['metadata']) for i, doc in enumerate(documents)]
index.upsert(upsert_data)
print("Documents upserted to the index with metadata.")


Documents upserted to the index with metadata.


In [28]:
user_query = "What is the content of document 1?"  # Example query
query_embedding = get_query_embedding(user_query)
results = index.query(vector=query_embedding, top_k=5, include_values=True, include_metadata=True)

# Print results with metadata
print("Top matching documents:")
for match in results['matches']:
    print(f"Document ID: {match['id']}, Score: {match['score']}, Metadata: {match.get('metadata', 'No metadata')}")


Top matching documents:
Document ID: 2, Score: 0.633944392, Metadata: No metadata
Document ID: 3, Score: 0.613308191, Metadata: No metadata
Document ID: 1, Score: 0.356235743, Metadata: {'author': 'Author B', 'title': 'Doc 2'}
Document ID: 0, Score: 0.312054545, Metadata: {'author': 'Author A', 'title': 'Doc 1'}


In [29]:
while True:
    user_query = input("Enter your query (or type 'exit' to quit): ")
    if user_query.lower() == 'exit':
        break

    query_embedding = get_query_embedding(user_query)
    results = index.query(vector=query_embedding, top_k=5, include_values=True, include_metadata=True)

    print("Top matching documents:")
    for match in results['matches']:
        print(f"Document ID: {match['id']}, Score: {match['score']}, Metadata: {match.get('metadata', 'No metadata')}")


Enter your query (or type 'exit' to quit): exit


In [30]:
# Loop to allow user interaction
while True:
    user_query = input("Enter your query (or type 'exit' to quit): ")
    if user_query.lower() == 'exit':
        print("Exiting the program.")
        break

    # Get the query embedding for the user's input
    query_embedding = get_query_embedding(user_query)

    # Query the Pinecone index
    results = index.query(vector=query_embedding, top_k=5, include_values=True, include_metadata=True)

    # Print results with metadata
    print("\nTop matching documents:")
    for match in results['matches']:
        print(f"Document ID: {match['id']}, Score: {match['score']}, Metadata: {match.get('metadata', 'No metadata')}")

    print()  # Add a newline for better readability


Enter your query (or type 'exit' to quit): exit
Exiting the program.


uploading documents


In [31]:
from google.colab import files

uploaded = files.upload()

# List uploaded files
uploaded_files = list(uploaded.keys())
print("Uploaded files:", uploaded_files)


Saving data science resume 2.pdf to data science resume 2.pdf
Uploaded files: ['data science resume 2.pdf']


preprocessing documents

In [32]:
import os

def load_documents(uploaded_files):
    documents = []
    for filename in uploaded_files:
        if filename.endswith('.txt'):  # Adjust for other formats if needed
            content = uploaded[filename].decode('utf-8')  # Use filename as a string
            documents.append({
                'id': filename,
                'text': content,
                'metadata': {'filename': filename}
            })
    return documents

documents = load_documents(uploaded_files)


generate embeddings

In [33]:
from transformers import BertTokenizer, BertModel
import torch

def get_embeddings(documents):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    embeddings = []
    for doc in documents:
        inputs = tokenizer(doc['text'], return_tensors='pt', truncation=True, padding=True)
        outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).detach().numpy()[0])  # Average pooling
    return embeddings

doc_embeddings = get_embeddings(documents)


upload to pinecone

In [34]:
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone with your API key
pc = Pinecone(api_key='1e7f6c93-018b-40f4-b83a-2ebb8bcd379e')

# Check if the index exists, if not create it
index_name = 'qa-bot-index'  # Replace with your index name
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,  # Adjust based on your embedding model's output dimension
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )

# Connect to the index
index = pc.Index(index_name)


In [35]:
user_query = "What is the content of document 1?"  # Example query
query_embedding = get_query_embedding(user_query)
results = index.query(vector=query_embedding, top_k=5, include_values=True, include_metadata=True)

# Print results
print("Top matching documents:")
for match in results['matches']:
    print(f"Document ID: {match['id']}, Score: {match['score']}, Metadata: {match.get('metadata', 'No metadata')}")


Top matching documents:
Document ID: 2, Score: 0.633944392, Metadata: No metadata
Document ID: 3, Score: 0.613308191, Metadata: No metadata
Document ID: 1, Score: 0.356235743, Metadata: {'author': 'Author B', 'title': 'Doc 2'}
Document ID: 0, Score: 0.312054545, Metadata: {'author': 'Author A', 'title': 'Doc 1'}


In [50]:
!pip install flask-cors


In [47]:
from flask import Flask, request, jsonify
from flask_cors import CORS
# from flask_ngrok import run_with_ngrok
import os
from pinecone import Pinecone

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes
# run_with_ngrok(app)  # Start ngrok when app is run

# Initialize Pinecone
pc = Pinecone(api_key='1e7f6c93-018b-40f4-b83a-2ebb8bcd379e')
index = pc.Index('qa-bot-index')

@app.route('/query', methods=['POST'])
def query():
    user_query = request.json.get('query')
    # Add logic to generate query embedding and get results
    results = index.query(vector=get_query_embedding(user_query), top_k=5, include_values=True)
    return jsonify(results)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [48]:
!ngrok authtoken 2mKvK9OHhoQiOnWBifnubjOhMhU_2eJmndwi3ZVLwUeHmn5oe

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [49]:
!pip install pyngrok

from pyngrok import ngrok
public_url = ngrok.connect(5000)
print("your public url:", public_url)

your public url: NgrokTunnel: "https://f143-34-139-231-254.ngrok-free.app" -> "http://localhost:5000"
